# DR7&MPL8

In [31]:

import import_ipynb
import cross_match
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo
from collections import OrderedDict

#############################################################################
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/pair-m-z/gal_info_dr7_v5_2.fit'
dir_indo = '/Users/astro/Documents/notebooks/zs/pair-m-z/cross-match_query_k_10.txt'
dir_k_10_not_enough = '/Users/astro/Documents/notebooks/zs/pair-m-z/cross_match_res/v2/cmatch_10_not_enough.txt'
dir_k_10_wrong='/Users/astro/Documents/notebooks/zs/pair-m-z/cross_match_res/v2/cmatch_10_wrong.txt'
dir_res = '/Users/astro/Documents/notebooks/zs/pair-m-z/cross_match_res/v2/'
############################################################################

def cross():
    
    with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7,\
        open(dir_indo, 'a+') as f, open(dir_k_10_not_enough, 'a+') as f_10,\
            open(dir_k_10_wrong,'a+') as f_10_w:

        data = f_drp[1].data
        manga = np.empty((len(data), 2), dtype=np.float64)
        manga[:, 0] = data['objra']
        manga[:, 1] = data['objdec']
        manga_z = data['z']
        plateifu = data['plateifu']

        data1 = f_dr7[1].data
        dr7 = np.empty((len(data1), 2), dtype=np.float64)
        dr7[:, 0] = data1['RA']
        dr7[:, 1] = data1['DEC']
        dr7_z = data1['Z']

        for i in range(0, len(manga)):
            print(plateifu[i])
            z = manga_z[i]
            dis = cosmo.comoving_distance(z)  # Mpc
            r = 0.15 / dis.value
            max_r = cross_match.r2arcsec(r)  # degree

            dist, ind = cross_match.crossmatch_angular(manga[i], dr7, max_r,10)
            match = ~np.isinf(dist)
            dist_match = dist[match]
            ind_match=ind[match]
            
            ind_1 = list(OrderedDict.fromkeys(list(ind_match)))
            match_1=[] # dist index which remove multiple elements
            for k in ind_1:
                index=np.where(ind_match==k)[0][0]
                match_1.append(dist_match[index])
#             print(plateifu[i],len(dist[0]),len(dist_match))
#             print(dist[0],dist_match)
            if len(ind_match) == len(ind_1) and len(ind_1)>0:
                if len(dist[0])==len(dist_match):
                    print(plateifu[i],manga[:,0][i],manga[:,1][i],max_r,file=f_10)
                elif len(dist[0])>len(dist_match):
                    with open(dir_res+plateifu[i]+'.txt','a+') as f_res:
                        print('index','ra','dec','min_dis_deg','max_r',file=f_res)
                        for j in range(0,len(ind_1)):
                            ra=dr7[:,0][ind_1[j]]
                            dec=dr7[:,1][ind_1[j]]
                            print(ind_1[j],ra,dec,match_1[j],max_r,file=f_res)
            elif len(ind_match)>len(ind_1) and len(ind_1)>0:
                if len(dist[0])==len(dist_match):
                    print(plateifu[i],manga[:,0][i],manga[:,1][i],max_r,file=f_10_w)
                elif len(dist[0])>len(dist_match):
                    with open(dir_res+plateifu[i]+'.txt','a+') as f_res:
                        print('index','ra','dec','min_dis_deg','max_r',file=f_res)
                        for j in range(0,len(ind_1)):
                            ra=dr7[:,0][ind_1[j]]
                            dec=dr7[:,1][ind_1[j]]
                            print(ind_1[j],ra,dec,match_1[j],max_r,file=f_res)


if __name__ == '__main__':
        
    cross()

10001-12701
10001-12702
10001-12703
10001-12704
10001-12705
10001-1901
10001-1902
10001-3701
10001-3702
10001-3703
10001-3704
10001-6101
10001-6102
10001-6103
10001-6104
10001-9101
10001-9102
10141-12701
10141-12702
10141-12703
10141-12704
10141-12705
10141-1901
10141-1902
10141-3701
10141-3702
10141-3703
10141-3704
10141-6101
10141-6102
10141-6103
10141-6104
10141-9101
10141-9102
10142-12701
10142-12702
10142-12703
10142-12704
10142-12705
10142-1901
10142-1902
10142-3701
10142-3702
10142-3703
10142-3704
10142-6101
10142-6102
10142-6103
10142-6104
10142-9101
10142-9102
10143-12701
10143-12702
10143-12703
10143-12704
10143-12705
10143-1901
10143-1902
10143-3701
10143-3702
10143-3703
10143-3704
10143-6101
10143-6102
10143-6103
10143-6104
10143-9101
10143-9102
10144-12701
10144-12702
10144-12703
10144-12704
10144-12705
10144-1901
10144-1902
10144-3701
10144-3702
10144-3703
10144-3704
10144-6101
10144-6102
10144-6103
10144-6104
10144-9101
10144-9102
10145-12701
10145-12702
10145-12703
1014

7960-12704
7960-12705
7960-1901
7960-1902
7960-3701
7960-3702
7960-3703
7960-3704
7960-6101
7960-6102
7960-6103
7960-6104
7960-9101
7960-9102
7962-12701
7962-12702
7962-12703
7962-12704
7962-12705
7962-1901
7962-1902
7962-3701
7962-3702
7962-3703
7962-3704
7962-6101
7962-6102
7962-6103
7962-6104
7962-9101
7962-9102
7964-12701
7964-12702
7964-12703
7964-12704
7964-12705
7964-1901
7964-1902
7964-3701
7964-3702
7964-3703
7964-3704
7964-6101
7964-6102
7964-6103
7964-6104
7964-9101
7964-9102
7968-12701
7968-12702
7968-12703
7968-12704
7968-12705
7968-1901
7968-1902
7968-3701
7968-3702
7968-3703
7968-3704
7968-6101
7968-6102
7968-6103
7968-6104
7968-9101
7968-9102
7972-12701
7972-12702
7972-12703
7972-12704
7972-12705
7972-1901
7972-1902
7972-3701
7972-3702
7972-3703
7972-3704
7972-6101
7972-6102
7972-6103
7972-6104
7972-9101
7972-9102
7975-12701
7975-12702
7975-12703
7975-12704
7975-12705
7975-1901
7975-1902
7975-3701
7975-3702
7975-3703
7975-3704
7975-6101
7975-6102
7975-6103
7975-6104
797

8239-12705
8239-1901
8239-1902
8239-3701
8239-3702
8239-3703
8239-3704
8239-6101
8239-6102
8239-6103
8239-6104
8239-9101
8239-9102
8241-12701
8241-12702
8241-12703
8241-12704
8241-12705
8241-1901
8241-1902
8241-3701
8241-3702
8241-3703
8241-3704
8241-6101
8241-6102
8241-6103
8241-6104
8241-9101
8241-9102
8243-12701
8243-12702
8243-12703
8243-12704
8243-12705
8243-1901
8243-1902
8243-3701
8243-3702
8243-3703
8243-3704
8243-6101
8243-6102
8243-6103
8243-6104
8243-9101
8243-9102
8244-12701
8244-12702
8244-12703
8244-12704
8244-12705
8244-1901
8244-1902
8244-3701
8244-3702
8244-3703
8244-3704
8244-6101
8244-6102
8244-6103
8244-6104
8244-9101
8244-9102
8245-12701
8245-12702
8245-12703
8245-12704
8245-12705
8245-1901
8245-1902
8245-3701
8245-3702
8245-3703
8245-3704
8245-6101
8245-6102
8245-6103
8245-6104
8245-9101
8245-9102
8247-12701
8247-12702
8247-12703
8247-12704
8247-12705
8247-1901
8247-1902
8247-3701
8247-3702
8247-3703
8247-3704
8247-6101
8247-6102
8247-6103
8247-6104
8247-9101
8247

8338-12703
8338-12704
8338-12705
8338-1901
8338-1902
8338-3701
8338-3702
8338-3703
8338-3704
8338-6101
8338-6102
8338-6103
8338-6104
8338-9101
8338-9102
8341-12701
8341-12702
8341-12703
8341-12704
8341-12705
8341-1901
8341-1902
8341-3701
8341-3702
8341-3703
8341-3704
8341-6101
8341-6102
8341-6103
8341-6104
8341-9101
8341-9102
8438-12701
8438-12702
8438-12703
8438-12704
8438-12705
8438-1901
8438-1902
8438-3701
8438-3702
8438-3703
8438-3704
8438-6101
8438-6102
8438-6103
8438-6104
8438-9101
8438-9102
8439-12701
8439-12702
8439-12703
8439-12704
8439-12705
8439-1901
8439-1902
8439-3701
8439-3702
8439-3703
8439-3704
8439-6101
8439-6102
8439-6103
8439-6104
8439-9101
8439-9102
8440-12701
8440-12702
8440-12703
8440-12704
8440-12705
8440-1901
8440-1902
8440-3701
8440-3702
8440-3703
8440-3704
8440-6101
8440-6102
8440-6103
8440-6104
8440-9101
8440-9102
8442-12701
8442-12702
8442-12703
8442-12704
8442-12705
8442-1901
8442-1902
8442-3701
8442-3702
8442-3703
8442-3704
8442-6101
8442-6102
8442-6103
84

8568-9102
8588-12701
8588-12702
8588-12703
8588-12704
8588-12705
8588-1901
8588-1902
8588-3701
8588-3702
8588-3703
8588-3704
8588-6101
8588-6102
8588-6103
8588-6104
8588-9101
8588-9102
8591-12701
8591-12702
8591-12703
8591-12704
8591-12705
8591-1901
8591-1902
8591-3701
8591-3702
8591-3703
8591-3704
8591-6101
8591-6102
8591-6103
8591-6104
8591-9101
8591-9102
8592-12701
8592-12702
8592-12703
8592-12704
8592-12705
8592-1901
8592-1902
8592-3701
8592-3702
8592-3703
8592-3704
8592-6101
8592-6102
8592-6103
8592-6104
8592-9101
8592-9102
8593-12701
8593-12702
8593-12703
8593-12704
8593-12705
8593-1901
8593-1902
8593-3701
8593-3702
8593-3703
8593-3704
8593-6101
8593-6102
8593-6103
8593-6104
8593-9101
8593-9102
8595-12701
8595-12702
8595-12703
8595-12704
8595-12705
8595-1901
8595-1902
8595-3701
8595-3702
8595-3703
8595-3704
8595-6101
8595-6102
8595-6103
8595-6104
8595-9101
8595-9102
8597-12701
8597-12702
8597-12703
8597-12704
8597-12705
8597-1901
8597-1902
8597-3701
8597-3702
8597-3703
8597-3704


8931-6103
8931-6104
8931-9101
8931-9102
8932-12701
8932-12702
8932-12703
8932-12704
8932-12705
8932-1901
8932-1902
8932-3701
8932-3702
8932-3703
8932-3704
8932-6101
8932-6102
8932-6103
8932-6104
8932-9101
8932-9102
8933-12701
8933-12702
8933-12703
8933-12704
8933-12705
8933-1901
8933-1902
8933-3701
8933-3702
8933-3703
8933-3704
8933-6101
8933-6102
8933-6103
8933-6104
8933-9101
8933-9102
8934-12701
8934-12702
8934-12703
8934-12704
8934-12705
8934-1901
8934-1902
8934-3701
8934-3702
8934-3703
8934-3704
8934-6101
8934-6102
8934-6103
8934-6104
8934-9101
8934-9102
8935-12701
8935-12702
8935-12703
8935-12704
8935-12705
8935-1901
8935-1902
8935-3701
8935-3702
8935-3703
8935-3704
8935-6101
8935-6102
8935-6103
8935-6104
8935-9101
8935-9102
8936-12701
8936-12702
8936-12703
8936-12704
8936-12705
8936-1901
8936-1902
8936-3701
8936-3702
8936-3703
8936-3704
8936-6101
8936-6102
8936-6103
8936-6104
8936-9101
8936-9102
8937-12701
8937-12702
8937-12703
8937-12704
8937-12705
8937-1901
8937-1902
8937-3701


9001-3704
9001-6101
9001-6102
9001-6103
9001-6104
9001-9101
9001-9102
9002-12701
9002-12702
9002-12703
9002-12704
9002-12705
9002-1901
9002-1902
9002-3701
9002-3702
9002-3703
9002-3704
9002-6101
9002-6102
9002-6103
9002-6104
9002-9101
9002-9102
9024-12701
9024-12702
9024-12703
9024-12704
9024-12705
9024-1901
9024-1902
9024-3701
9024-3702
9024-3703
9024-3704
9024-6101
9024-6102
9024-6103
9024-6104
9024-9101
9024-9102
9025-12701
9025-12702
9025-12703
9025-12704
9025-12705
9025-1901
9025-1902
9025-3701
9025-3702
9025-3703
9025-3704
9025-6101
9025-6102
9025-6103
9025-6104
9025-9101
9025-9102
9026-12701
9026-12702
9026-12703
9026-12704
9026-12705
9026-1901
9026-1902
9026-3701
9026-3702
9026-3703
9026-3704
9026-6101
9026-6102
9026-6103
9026-6104
9026-9101
9026-9102
9027-12701
9027-12702
9027-12703
9027-12704
9027-12705
9027-1901
9027-1902
9027-3701
9027-3702
9027-3703
9027-3704
9027-6101
9027-6102
9027-6103
9027-6104
9027-9101
9027-9102
9028-12701
9028-12702
9028-12703
9028-12704
9028-12705


9189-3701
9189-3702
9189-3703
9189-3704
9189-6101
9189-6102
9189-6103
9189-6104
9189-9101
9189-9102
9190-12701
9190-12702
9190-12703
9190-12704
9190-12705
9190-1901
9190-1902
9190-3701
9190-3702
9190-3703
9190-3704
9190-6101
9190-6102
9190-6103
9190-6104
9190-9101
9190-9102
9192-12701
9192-12702
9192-12703
9192-12704
9192-12705
9192-1901
9192-1902
9192-3701
9192-3702
9192-3703
9192-3704
9192-6101
9192-6102
9192-6103
9192-6104
9192-9101
9192-9102
9193-12701
9193-12702
9193-12703
9193-12704
9193-12705
9193-1901
9193-1902
9193-3701
9193-3702
9193-3703
9193-3704
9193-6101
9193-6102
9193-6103
9193-6104
9193-9101
9193-9102
9194-12701
9194-12702
9194-12703
9194-12704
9194-12705
9194-1901
9194-1902
9194-3701
9194-3702
9194-3703
9194-3704
9194-6101
9194-6102
9194-6103
9194-6104
9194-9101
9194-9102
9195-12701
9195-12702
9195-12703
9195-12704
9195-12705
9195-1901
9195-1902
9195-3701
9195-3702
9195-3703
9195-3704
9195-6101
9195-6102
9195-6103
9195-6104
9195-9101
9195-9102
9196-12701
9196-12702
919

9867-1901
9867-1902
9867-3701
9867-3702
9867-3703
9867-3704
9867-6101
9867-6102
9867-6103
9867-6104
9867-9101
9867-9102
9868-12701
9868-12702
9868-12703
9868-12704
9868-12705
9868-1901
9868-1902
9868-3701
9868-3702
9868-3703
9868-3704
9868-6101
9868-6102
9868-6103
9868-6104
9868-9101
9868-9102
9869-12701
9869-12702
9869-12703
9869-12704
9869-12705
9869-1901
9869-1902
9869-3701
9869-3702
9869-3703
9869-3704
9869-6101
9869-6102
9869-6103
9869-6104
9869-9101
9869-9102
9870-12701
9870-12702
9870-12703
9870-12704
9870-12705
9870-1901
9870-1902
9870-3701
9870-3702
9870-3703
9870-3704
9870-6101
9870-6102
9870-6103
9870-6104
9870-9101
9870-9102
9871-12701
9871-12702
9871-12703
9871-12704
9871-12705
9871-1901
9871-1902
9871-3701
9871-3702
9871-3703
9871-3704
9871-6101
9871-6102
9871-6103
9871-6104
9871-9101
9871-9102
9872-12701
9872-12702
9872-12703
9872-12704
9872-12705
9872-1901
9872-1902
9872-3701
9872-3702
9872-3703
9872-3704
9872-6101
9872-6102
9872-6103
9872-6104
9872-9101
9872-9102
9873-

# cross_match_395_k>10

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import OrderedDict

import import_ipynb
import cross_match
from astropy.cosmology import WMAP9 as cosmo
from astropy.io import fits
from astropy.table import Table



#############################################################################
dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dr7 = '/Users/astro/Documents/notebooks/zs/data/gal_info_dr7_v5_2.fit'
dir_k_10_not_enough = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v2/cmatch_10_not_enough.txt'
dir_res = '/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v2/'
############################################################################


def cross():

    with fits.open(dir_drp) as f_drp, fits.open(dir_dr7) as f_dr7:

        f_10 = Table.read(dir_k_10_not_enough, format='ascii')
        manga = np.empty((len(f_10), 2), dtype=np.float64)
        manga[:, 0] = f_10['col2']
        manga[:, 1] = f_10['col3']
        plateifu = f_10['col1']

        data = f_drp[1].data
        Z = data['Z']
        pifu = data['plateifu']

        data1 = f_dr7[1].data
        dr7 = np.empty((len(data1), 2), dtype=np.float64)
        dr7[:, 0] = data1['RA']
        dr7[:, 1] = data1['DEC']
        dr7_z = data1['Z']

        for i in range(0, len(manga)):

            index = np.where(pifu == plateifu[i])[0]
            z = Z[index]
            dis = cosmo.comoving_distance(z)  # Mpc
            r = 0.15 / dis.value
            max_r = cross_match.r2arcsec(r)  # degree

            dist, ind = cross_match.crossmatch_angular(manga[i], dr7,20, max_r)
            match = ~np.isinf(dist)
            dist_match = dist[match]
            ind_match = ind[match]

            ind_1 = list(OrderedDict.fromkeys(list(ind_match)))
            match_1 = []  # dist index which remove multiple elements
            for k in ind_1:
                index = np.where(ind_match == k)[0][0]
                match_1.append(dist_match[index])

            if len(ind_match) == len(ind_1) and len(ind_1) > 0:
                if len(dist[0]) == len(dist_match):
                    print('20 is not enough', plateifu[i], manga[:, 0][i],
                          manga[:, 1][i], max_r)
                elif len(dist[0]) > len(dist_match):
                    with open(dir_res + plateifu[i] + '.txt', 'a+') as f_res:
                        print('index',
                              'ra',
                              'dec',
                              'min_dis_deg',
                              'max_r',
                              file=f_res)
                        for j in range(0, len(ind_1)):
                            ra = dr7[:, 0][ind_1[j]]
                            dec = dr7[:, 1][ind_1[j]]
                            print(ind_1[j],
                                  ra,
                                  dec,
                                  match_1[j],
                                  max_r,
                                  file=f_res)

if __name__ == '__main__':

    cross()

importing Jupyter notebook from cross_match.ipynb
20 is not enough 10001-12704 133.989966869 57.6779676669 [0.14115479]
20 is not enough 10507-1902 176.525541667 47.4820555556 [0.65845641]
20 is not enough 10509-12703 168.545376088 48.318515932 [0.27914476]
20 is not enough 10510-12704 179.117267145 55.1252090259 [0.53556613]
20 is not enough 10517-12702 149.99409 5.32659 [1.97975657]
20 is not enough 7958-6101 257.383688857 34.4270318778 [0.08361879]
20 is not enough 8078-3703 42.3874325272 -0.784442553068 [0.0842864]
20 is not enough 8079-1901 42.407980379 -0.813857447855 [0.08140749]
20 is not enough 8079-6101 43.1562184278 -1.16906729877 [0.08903433]
20 is not enough 8080-12702 47.9994154757 -1.16166126075 [0.07460794]
20 is not enough 8080-12705 47.8890844551 -0.742939934572 [0.0708262]
20 is not enough 8080-1901 47.9851588295 -0.0325628998893 [0.05324287]
20 is not enough 8080-3701 47.528362637 -0.468033748749 [0.05456005]
20 is not enough 8080-3704 49.4574544521 -0.554658522356 

In [2]:
from astropy.table import Table

t=Table.read('/Users/astro/Documents/notebooks/zs/cross_match/cross_match_res/v2/cmatch_10_not_enough.txt',format='ascii')
print(t,len(t))

    col1         col2          col3             col4        
----------- ------------- ------------- --------------------
10001-12704 133.989966869 57.6779676669  0.14115478769125356
 10001-3701 132.465646765 57.1437279024 0.050848013169786446
 10001-3702 132.912768243 57.1074235568  0.07803981454301591
 10001-6102 132.653991697 57.3596675436  0.07655848771578087
 10213-3703      123.8433      44.06446  0.09834188009019654
 10219-1901  125.14376637 21.1054663161   0.1182713927146769
 10219-1902 125.062375384 21.0600550691  0.11999492813396592
 10219-6102 125.148697904 21.0677945565   0.1325546782566985
10221-12703     124.05695        25.076  0.12770682318244184
10221-12705 124.109635921 23.1425158913  0.14425187053577215
        ...           ...           ...                  ...
  9877-1902 195.270098514 27.8917529314  0.10310391900048708
  9877-3704 194.662274497 27.9539019989  0.09985051496205223
  9877-6104 195.090290656 27.8985596385  0.12265470861052226
  9879-1901 197.34533081